In [60]:
import numpy as np
import pandas as pd
import re
import requests

## 병원 데이터 전처리

In [61]:
# 병원 풀데이터 불러오기
hos = pd.read_excel("D:/d드라이브/00_민경자료/01_데이터사이언스학부/4학년 2025/1학기/캡스톤 디자인/병원 풀데이터.xlsx")

# 데이터프레임 형태로 변경
hos_df = pd.DataFrame(hos)

print(hos_df.shape)

(77584, 34)


In [62]:
# 병원 풀데이터에서 필요 없는 변수제거
drop_cols = ['dutyEmcls', 'dutyEmclsName', 'dutyEryn', 'dutyInf', 'dutyMapimg',
             'dutyName', 'dutyTel1', 'dutyTime1c', 'dutyTime1s', 'dutyTime2c',
             'dutyTime2s', 'dutyTime3c', 'dutyTime3s', 'dutyTime4c', 'dutyTime4s',
             'dutyTime5c', 'dutyTime5s', 'hpid', 'postCdn1', 'postCdn2',
             'rnum', 'dutyTime6c', 'dutyTime6s', 'dutyTime7c', 'dutyTime7s',
             'dutyTime8c', 'dutyTime8s', 'dutyEtc', 'dutyTel3']

# 필요한하는 컬럼만 추려서 삭제
drop_cols_exist = [col for col in drop_cols if col in hos_df.columns]

hos_df = hos_df.drop(columns=drop_cols_exist)

In [63]:
# 각 변수 결측값 개수 확인
print(hos_df.isnull().sum())

dutyAddr      0
dutyDiv       0
dutyDivNam    0
wgs84Lat      3
wgs84Lon      3
dtype: int64


In [64]:
# 지도 시각화를 위해 경도나 위도가 결측값인 행 삭제
hos_df = hos_df.dropna(subset=['wgs84Lat', 'wgs84Lon']).reset_index(drop=True)

# 각 변수 결측값 개수 확인
print(hos_df.isnull().sum())
print(hos_df.shape)

dutyAddr      0
dutyDiv       0
dutyDivNam    0
wgs84Lat      0
wgs84Lon      0
dtype: int64
(77581, 5)


In [65]:
# 시도/시군구/읍면동 구분을 위한 사용자 함수 정의
def parse_address_with_brackets(addr):
    try:
        eup_myeon_dong = None

        # 1. 괄호 안에서 읍면동 찾기
        match = re.search(r'\(([^)]+)\)', addr)
        if match:
            for word in match.group(1).split():
                word_cleaned = re.sub(r'[^\w가-힣]', '', word)  # 특수문자 제거
                if any(x in word_cleaned for x in ['읍', '면', '동', '가']):
                    eup_myeon_dong = word_cleaned
                    break

        # 2. 전체 주소에서도 읍면동 찾기
        if not eup_myeon_dong:
            for word in addr.split():
                word_cleaned = re.sub(r'[^\w가-힣]', '', word)  # 특수문자 제거
                if any(x in word_cleaned for x in ['읍', '면', '동', '가']):
                    eup_myeon_dong = word_cleaned
                    break

        parts = addr.split()
        시도 = None
        시군구_list = []

        # 시도 변환용 매핑
        sido_mapping = {
            '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
            '인천': '인천광역시', '광주': '광주광역시', '대전': '대전광역시',
            '울산': '울산광역시', '세종': '세종특별자치시', '경기': '경기도',
            '강원': '강원특별자치도', '충북': '충청북도', '충남': '충청남도',
            '전북': '전북특별자치도', '전남': '전라남도', '경북': '경상북도',
            '경남': '경상남도', '제주': '제주특별자치도'
        }

        for part in parts:
            if 시도 is None:
                if part in sido_mapping.values():
                    시도 = part
                    continue
                elif part in sido_mapping:
                    시도 = sido_mapping[part]
                    continue
                else:
                    for short, full in sido_mapping.items():
                        if part.startswith(short):
                            시도 = full
                            break
                if 시도:
                    continue

            if part.endswith(('시', '군', '구')) and len(시군구_list) < 2:
                시군구_list.append(part)

        시군구 = ' '.join(시군구_list) if 시군구_list else ('' if 시도 == '세종특별자치시' else None)

        # ✅ 지번주소 생성
        지번 = ' '.join(filter(None, [시도, 시군구, eup_myeon_dong]))

        return pd.Series([시도, 시군구, eup_myeon_dong, 지번])

    except:
        return pd.Series([None, None, None, None])

In [66]:
hos_df[['시도', '시군구', '읍면동','지번']] = hos_df['dutyAddr'].apply(parse_address_with_brackets)

In [67]:
# 병원 데이터에 결측값이 있는지 확인하기
print(hos_df['시도'].isnull().sum())
print(hos_df['시군구'].isnull().sum())
print(hos_df['읍면동'].isnull().sum())

0
8
875


In [68]:
# 병원 데이터의 결측값 확인해보기
a = hos_df[hos_df['읍면동'].isnull()]
print(a['dutyAddr'].head(10))

21          제주특별자치도 제주시 연북로 38
132       충청북도 청주시 상당구 월평로 188
135      충청북도 청주시 청원구 충청대로 121
175           경상남도 양산시 덕계로 117
282      경상남도 창원시 마산합포구 합포로 42
363            경상남도 진주시 장대로 12
376         전북특별자치도 군산시 백릉로 44
418    전북특별자치도 전주시 완산구 팔달로 232
587        경상남도 김해시 함박로101번길 8
654            전라남도 목포시 석현로 48
Name: dutyAddr, dtype: object


도로명 주소로 인해 결측치 발생

In [69]:
# 카카오 Map API를 활용해 도록명 주소를 지번 주소로 변경

# 카카오 Map에서 발급받은 REST API KEY
KAKAO_API_KEY = "9c32bb7527635581ea8a635372d899e1"

# 역지오코딩: 위경도 → 주소
def reverse_geocode(lat, lon, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": lon, "y": lat}
    try:
        res = requests.get(url, headers=headers, params=params)
        res.raise_for_status()
        result = res.json()
        return result['documents'][0]['address']['address_name']
    except:
        return None
    
# 주소 문자열에서 시도, 시군구, 읍면동 추출
def extract_admin_parts(addr):
    try:
        if pd.isnull(addr) or not isinstance(addr, str):
            return pd.Series([None, None, None])

        parts = addr.split()
        시도 = None
        시군구_list = []
        읍면동 = None

        # 시도 매핑
        sido_mapping = {
            '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
            '인천': '인천광역시', '광주': '광주광역시', '대전': '대전광역시',
            '울산': '울산광역시', '세종': '세종특별자치시', '경기': '경기도',
            '강원': '강원특별자치도', '충북': '충청북도', '충남': '충청남도',
            '전북': '전북특별자치도', '전남': '전라남도', '경북': '경상북도',
            '경남': '경상남도', '제주': '제주특별자치도'
        }

        # 시도 추출
        for part in parts:
            if part in sido_mapping.values():
                시도 = part
                break
            elif part in sido_mapping:
                시도 = sido_mapping[part]
                break
            else:
                for short, full in sido_mapping.items():
                    if part.startswith(short):
                        시도 = full
                        break
            if 시도:
                break

        # 시군구 추출
        for part in parts[1:]:
            if part.endswith(('시', '군', '구')):
                시군구_list.append(part)
                if len(시군구_list) == 2:
                    break
        시군구 = ' '.join(시군구_list) if 시군구_list else (None if 시도 != '세종특별자치시' else '')

        # 읍면동 추출 (괄호 안 포함 고려)
        bracket_match = re.search(r'\(([^)]+)\)', addr)
        if bracket_match:
            for word in bracket_match.group(1).split():
                if any(x in word for x in ['읍', '면', '동', '가']):
                    읍면동 = word
                    break

        if not 읍면동:
            for word in parts:
                if any(x in word for x in ['읍', '면', '동', '가']):
                    읍면동 = word
                    break

        return pd.Series([시도, 시군구, 읍면동])

    except:
        return pd.Series([None, None, None])

In [70]:
# 1. 보완 대상: 시도/시군구/읍면동 중 하나라도 결측, 위경도는 존재하는 행
target_df = hos_df[
    (hos_df['시도'].isnull() | hos_df['시군구'].isnull() | hos_df['읍면동'].isnull()) &
    (hos_df['wgs84Lat'].notnull() & hos_df['wgs84Lon'].notnull())
].copy()

# 2. 역지오코딩 주소 가져오기
target_df['역지오코딩_주소'] = target_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 3. 역지오코딩 주소에서 시도/시군구/읍면동 추출
target_df[['시도_보완', '시군구_보완', '읍면동_보완']] = target_df['역지오코딩_주소'].apply(extract_admin_parts)


In [71]:
# 4. 병합 + 보완값으로 채우기
hos_df = hos_df.merge(
    target_df[['dutyAddr', '시도_보완', '시군구_보완', '읍면동_보완']],
    on='dutyAddr',
    how='left'
)
hos_df['시도'] = hos_df['시도'].fillna(hos_df['시도_보완'])
hos_df['시군구'] = hos_df['시군구'].fillna(hos_df['시군구_보완'])
hos_df['읍면동'] = hos_df['읍면동'].fillna(hos_df['읍면동_보완'])
hos_df.drop(columns=['시도_보완', '시군구_보완', '읍면동_보완'], inplace=True)

In [72]:
# 역지오코딩 완료 후 결측값 수 확인
print("시도 결측 수:", hos_df['시도'].isnull().sum())
print("시군구 결측 수:", hos_df['시군구'].isnull().sum())
print("읍면동 결측 수:", hos_df['읍면동'].isnull().sum())

# 결측이 존재하는 행 중에서 위경도 있는 행만 필터링
still_missing = hos_df[
    (hos_df['시도'].isnull() | hos_df['시군구'].isnull() | hos_df['읍면동'].isnull()) &
    (hos_df['wgs84Lat'].notnull() & hos_df['wgs84Lon'].notnull())
].copy()

# 주소만 비교용 병합
compare_address_only = still_missing.merge(
    target_df[['dutyAddr', '역지오코딩_주소']],
    on='dutyAddr',
    how='left'
)

# 주소만 출력
print("\n결측 남은 주소의 역지오코딩 결과 (주소만 비교):\n")
print(compare_address_only[['dutyAddr', '역지오코딩_주소']])

시도 결측 수: 0
시군구 결측 수: 0
읍면동 결측 수: 12

결측 남은 주소의 역지오코딩 결과 (주소만 비교):

                              dutyAddr           역지오코딩_주소
0   서울특별시 종로구 세종대로 159, 7층 (세종로, 세광빌딩)   서울 종로구 세종로 202-1
1         서울특별시 종로구 세종대로 159, 8층 (세종로)   서울 종로구 세종로 202-1
2         서울특별시 종로구 세종대로 159, 9층 (세종로)   서울 종로구 세종로 202-1
3         서울특별시 종로구 세종대로 159, 5층 (세종로)   서울 종로구 세종로 202-1
4         서울특별시 종로구 세종대로 159, 6층 (세종로)   서울 종로구 세종로 202-1
5        서울특별시 종로구 세종대로23길 7, 3층 (세종로)   서울 종로구 세종로 161-1
6              대구광역시 중구 남성로 36-1 (남성로)    대구 중구 남성로 105-3
7    대구광역시 중구 달구벌대로415길 34, 1,2층 (남성로)       대구 중구 남성로 60
8                 대구광역시 중구 남성로 2 (남성로)      대구 중구 남성로 149
9                대구광역시 중구 남성로 44 (남성로)     대구 중구 남성로 99-1
10          대구광역시 중구 달성로 76, 1층 (시장북로)  대구 중구 시장북로 152-22
11               대구광역시 중구 남성로 10 (남성로)    대구 중구 남성로 132-1


도로명 주소를 카카오 Map API를 활용해 지번주소로 변경하려 했지만 변경되지 않는 행 삭제

In [73]:
# 병원 데이터에 결측값이 있는지 확인하기
print(hos_df['시도'].isnull().sum())
print(hos_df['시군구'].isnull().sum())
print(hos_df['읍면동'].isnull().sum())
print(hos_df.shape)

0
0
12
(77663, 9)


In [74]:
# 시도, 시군구, 읍면동 중 하나라도 결측인 행 삭제
hos_df = hos_df.dropna(subset=['시도', '시군구', '읍면동']).reset_index(drop=True)
print(hos_df.shape)

(77651, 9)


## 약국 데이터 전처리

In [75]:
# 약국 풀데이터 불러오기
ph = pd.read_excel("D:/d드라이브/00_민경자료/01_데이터사이언스학부/4학년 2025/1학기/캡스톤 디자인/약국 풀데이터.xlsx")

# 데이터프레임 형태로 변경
ph_df = pd.DataFrame(ph)

print(ph_df.shape)

(24847, 29)


In [76]:
ph_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24847 entries, 0 to 24846
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dutyAddr    24847 non-null  object 
 1   dutyFax     10251 non-null  object 
 2   dutyName    24847 non-null  object 
 3   dutyTel1    24847 non-null  object 
 4   dutyTime1c  24773 non-null  float64
 5   dutyTime1s  24773 non-null  float64
 6   dutyTime2c  24751 non-null  float64
 7   dutyTime2s  24751 non-null  float64
 8   dutyTime3c  24715 non-null  float64
 9   dutyTime3s  24715 non-null  float64
 10  dutyTime4c  24728 non-null  float64
 11  dutyTime4s  24728 non-null  float64
 12  dutyTime5c  24756 non-null  float64
 13  dutyTime5s  24756 non-null  float64
 14  dutyTime6c  22370 non-null  float64
 15  dutyTime6s  22370 non-null  float64
 16  hpid        24847 non-null  object 
 17  postCdn1    24846 non-null  float64
 18  postCdn2    24846 non-null  float64
 19  rnum        24847 non-nul

In [77]:
# 약국 풀데이터에서 필요 없는 변수제거
drop_cols = ['dutyEtc', 'dutyName', 'dutyFax', 'dutyInf', 'dutyMapimg','dutyTel1', 
             'dutyTime1c','dutyTime2c','dutyTime3c','dutyTime4c','dutyTime5c','dutyTime6c','dutyTime7c','dutyTime8c',
             'dutyTime1s','dutyTime2s','dutyTime3s','dutyTime4s','dutyTime5s','dutyTime6s','dutyTime7s','dutyTime8s',
             'rnum','hpid','postCdn1','postCdn2']

# 필요한하는 컬럼만 추려서 삭제
drop_cols_exist = [col for col in drop_cols if col in ph_df.columns]

ph_df = ph_df.drop(columns=drop_cols_exist)

In [78]:
# 각 변수 결측값 개수 확인
print(ph_df.isnull().sum())

dutyAddr    0
wgs84Lat    2
wgs84Lon    2
dtype: int64


In [79]:
# 지도 시각화를 위해 경도나 위도가 결측값인 행 삭제
ph_df = ph_df.dropna(subset=['wgs84Lat', 'wgs84Lon']).reset_index(drop=True)

# 각 변수 결측값 개수 확인
print(ph_df.isnull().sum())
print(ph_df.shape)

dutyAddr    0
wgs84Lat    0
wgs84Lon    0
dtype: int64
(24845, 3)


In [80]:
ph_df[['시도', '시군구', '읍면동','지번']] = ph_df['dutyAddr'].apply(parse_address_with_brackets)

In [81]:
# 병원 데이터에 결측값이 있는지 확인하기
print(ph_df['시도'].isnull().sum())
print(ph_df['시군구'].isnull().sum())
print(ph_df['읍면동'].isnull().sum())

0
0
2030


In [82]:
# 병원 데이터의 결측값 확인해보기
a = ph_df[ph_df['읍면동'].isnull()]
print(a['dutyAddr'].head(10))

10             서울특별시 서대문구 응암로 53
22           서울특별시 관악구 관악로30길 12
30              서울특별시 마포구 도화길 32
89          인천광역시 부평구 일신로14번길 10
119        서울특별시 영등포구 국제금융로6길 33
159           서울특별시 서대문구 연희맛로 28
160       서울특별시 서대문구 통일로40안길 3-1
185            서울특별시 양천구 오목로 299
190            서울특별시 중구 서소문로 109
207    서울특별시 서대문구 통일로 131 ,4층 2호
Name: dutyAddr, dtype: object


도로명 주소로 인해 결측치 발생

In [83]:
# 1. 보완 대상: 시도/시군구/읍면동 중 하나라도 결측, 위경도는 존재하는 행
target_df = ph_df[
    (ph_df['시도'].isnull() | ph_df['시군구'].isnull() | ph_df['읍면동'].isnull()) &
    (ph_df['wgs84Lat'].notnull() & ph_df['wgs84Lon'].notnull())
].copy()

# 2. 역지오코딩 주소 가져오기
target_df['역지오코딩_주소'] = target_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 3. 역지오코딩 주소에서 시도/시군구/읍면동 추출
target_df[['시도_보완', '시군구_보완', '읍면동_보완']] = target_df['역지오코딩_주소'].apply(extract_admin_parts)


In [84]:
# 4. 병합 + 보완값으로 채우기
ph_df = ph_df.merge(
    target_df[['dutyAddr', '시도_보완', '시군구_보완', '읍면동_보완']],
    on='dutyAddr',
    how='left'
)
ph_df['시도'] = ph_df['시도'].fillna(ph_df['시도_보완'])
ph_df['시군구'] = ph_df['시군구'].fillna(ph_df['시군구_보완'])
ph_df['읍면동'] = ph_df['읍면동'].fillna(ph_df['읍면동_보완'])
ph_df.drop(columns=['시도_보완', '시군구_보완', '읍면동_보완'], inplace=True)

In [85]:
# 역지오코딩 완료 후 결측값 수 확인
print("시도 결측 수:", ph_df['시도'].isnull().sum())
print("시군구 결측 수:", ph_df['시군구'].isnull().sum())
print("읍면동 결측 수:", ph_df['읍면동'].isnull().sum())

# 결측이 존재하는 행 중에서 위경도 있는 행만 필터링
still_missing = ph_df[
    (ph_df['시도'].isnull() | ph_df['시군구'].isnull() | ph_df['읍면동'].isnull()) &
    (ph_df['wgs84Lat'].notnull() & ph_df['wgs84Lon'].notnull())
].copy()

# 주소만 비교용 병합
compare_address_only = still_missing.merge(
    target_df[['dutyAddr', '역지오코딩_주소']],
    on='dutyAddr',
    how='left'
)

# 주소만 출력
print("\n결측 남은 주소의 역지오코딩 결과 (주소만 비교):\n")
print(compare_address_only[['dutyAddr', '역지오코딩_주소']])

시도 결측 수: 0
시군구 결측 수: 0
읍면동 결측 수: 4

결측 남은 주소의 역지오코딩 결과 (주소만 비교):

                         dutyAddr          역지오코딩_주소
0               서울특별시 서대문구 응암로 53              None
1             서울특별시 관악구 관악로30길 12              None
2    서울특별시 종로구 세종대로 159, 4층 (세종로)  서울 종로구 세종로 202-1
3  대구광역시 중구 중앙대로81길 105, 1층 (남성로)     대구 중구 남성로 8-3


In [86]:
# 약국국 데이터에 결측값이 있는지 확인하기
print(ph_df['시도'].isnull().sum())
print(ph_df['시군구'].isnull().sum())
print(ph_df['읍면동'].isnull().sum())
print(ph_df.shape)

0
0
4
(24903, 7)


In [87]:
# 시도, 시군구, 읍면동 중 하나라도 결측인 행 삭제
ph_df = ph_df.dropna(subset=['시도', '시군구', '읍면동']).reset_index(drop=True)
print(ph_df.shape)

(24899, 7)


## 인구 데이터 전처리

2025년 3월 인구 데이터 현황

In [88]:
# 인구 데이터 불러오기
pl = pd.read_excel("D:/d드라이브/00_민경자료/01_데이터사이언스학부/4학년 2025/1학기/캡스톤 디자인/202503_연령별인구현황_시군구.xlsx")

# 데이터프레임 형태로 변경
pl_df = pd.DataFrame(pl)

print(pl_df.shape)

(3630, 15)


In [89]:
# 인구 풀데이터에서 필요 없는 변수제거
drop_cols = ['행정기관코드']

# 필요한하는 컬럼만 추려서 삭제
drop_cols_exist = [col for col in drop_cols if col in pl_df.columns]

pl_df = pl_df.drop(columns=drop_cols_exist)

In [90]:
pl_df[['시도', '시군구', '읍면동']] = pl_df['행정기관'].apply(parse_address_with_brackets).iloc[:, :3]
pl_df = pl_df.rename(columns={'행정기관': '지번'})

## 이제 뭘 해볼까..?

In [91]:
hos_df

,dutyAddr,dutyDiv,dutyDivNam,wgs84Lat,wgs84Lon,시도,시군구,읍면동,지번
0,전북특별자치도 무주군 무주읍 주계로8길 6,N,치과의원,36.006966,127.661907,전북특별자치도,무주군,무주읍,전북특별자치도 무주군 무주읍
1,전북특별자치도 무주군 무주읍 단천로 37,C,의원,36.006068,127.656283,전북특별자치도,무주군,무주읍,전북특별자치도 무주군 무주읍
2,전북특별자치도 임실군 오수면 삼일로 11 (오수면),C,의원,35.539514,127.327056,전북특별자치도,임실군,오수면,전북특별자치도 임실군 오수면
3,전북특별자치도 임실군 관촌면 사선로 56 (관촌면),C,의원,35.675364,127.269897,전북특별자치도,임실군,관촌면,전북특별자치도 임실군 관촌면
4,전북특별자치도 임실군 임실읍 봉황로 203 (다사랑의원),C,의원,35.618100,127.282928,전북특별자치도,임실군,임실읍,전북특별자치도 임실군 임실읍
...,...,...,...,...,...,...,...,...,...
77646,전북특별자치도 전주시 완산구 백제대로 52 (삼천동1가),N,치과의원,35.799243,127.129364,전북특별자치도,전주시 완산구,삼천동1가,전북특별자치도 전주시 완산구 삼천동1가
77647,"전북특별자치도 완주군 삼례읍 삼봉8로 10-10, 완주군보건소",R,보건소,35.921136,127.092639,전북특별자치도,완주군,삼례읍,전북특별자치도 완주군 삼례읍
77648,전북특별자치도 전주시 덕진구 안덕원로 166-1 (인후동2가),N,치과의원,35.833655,127.146304,전북특별자치도,전주시 덕진구,인후동2가,전북특별자치도 전주시 덕진구 인후동2가
77649,"전북특별자치도 전주시 덕진구 호성로 132, 201,202호 (호성동1가, 진흥더블...",N,치과의원,35.863602,127.148341,전북특별자치도,전주시 덕진구,호성동1가,전북특별자치도 전주시 덕진구 호성동1가


In [92]:
ph_df

,dutyAddr,wgs84Lat,wgs84Lon,시도,시군구,읍면동,지번
0,"경상남도 진주시 범골로 9, 1층 103호 (충무공동)",35.165743,128.128769,경상남도,진주시,충무공동,경상남도 진주시 충무공동
1,"서울특별시 마포구 백범로 199, 메트로디오빌 103A호 (신공덕동)",37.543530,126.952771,서울특별시,마포구,신공덕동,서울특별시 마포구 신공덕동
2,"서울특별시 마포구 백범로 202, 101호 (신공덕동, 마포케이씨씨웰츠타워)",37.542848,126.952469,서울특별시,마포구,신공덕동,서울특별시 마포구 신공덕동
3,"서울특별시 강남구 일원로 지하121, 일원역 지하M층 338-M01호 (일원동)",37.484074,127.084452,서울특별시,강남구,일원동,서울특별시 강남구 일원동
4,"서울특별시 강남구 테헤란로 지하156, 역삼역 2호선 지하상가 127호 (역삼동)",37.500716,127.036618,서울특별시,강남구,역삼동,서울특별시 강남구 역삼동
...,...,...,...,...,...,...,...
24894,"전라남도 광양시 광양읍 신재로 18, 1층",34.972751,127.589496,전라남도,광양시,광양읍,전라남도 광양시 광양읍
24895,"대전광역시 서구 관저동로 164, 1층 (관저동)",36.306888,127.341140,대전광역시,서구,관저동,대전광역시 서구 관저동
24896,"대전광역시 동구 대전로542번길 80-87, 103,104호 (천동)",36.316461,127.447403,대전광역시,동구,천동,대전광역시 동구 천동
24897,"대전광역시 중구 선화서로 76, 102호 (선화동)",36.330356,127.416944,대전광역시,중구,선화동,대전광역시 중구 선화동


In [93]:
pl_df

,지번,총 거주자수,연령구간인구수,0~9세,10~19세,20~29세,30~39세,40~49세,50~59세,60~69세,70~79세,80~89세,90~99세,100세 이상,시도,시군구,읍면동
0,서울특별시 종로구 청운효자동,"10,866","10,866",532,"1,016","1,284","1,492","1,742","1,862","1,372",909,559,96,2,서울특별시,종로구,청운효자동
1,서울특별시 종로구 사직동,"8,792","8,792",395,616,"1,028","1,419","1,354","1,424","1,233",711,503,102,7,서울특별시,종로구,사직동
2,서울특별시 종로구 삼청동,"2,157","2,157",80,152,229,274,297,330,361,253,149,32,0,서울특별시,종로구,삼청동
3,서울특별시 종로구 부암동,"8,894","8,894",354,774,"1,020","1,181","1,312","1,575","1,356",792,440,87,3,서울특별시,종로구,부암동
4,서울특별시 종로구 평창동,"16,967","16,967",915,"1,508","1,868","2,000","2,374","3,013","2,778","1,489",821,189,12,서울특별시,종로구,평창동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625,제주특별자치도 서귀포시 서홍동,"11,060","11,060","1,030","1,502","1,084","1,161","1,864","1,900","1,305",779,370,64,1,제주특별자치도,서귀포시,서홍동
3626,제주특별자치도 서귀포시 대륜동,"14,940","14,940","1,237","1,397","1,251","1,916","2,591","2,544","2,109","1,211",572,106,6,제주특별자치도,서귀포시,대륜동
3627,제주특별자치도 서귀포시 대천동,"13,612","13,612","1,276","1,377","1,195","1,768","2,459","2,125","1,806","1,033",478,94,1,제주특별자치도,서귀포시,대천동
3628,제주특별자치도 서귀포시 중문동,"12,069","12,069",838,"1,116","1,220","1,553","2,047","2,064","1,676",930,508,115,2,제주특별자치도,서귀포시,중문동


In [94]:
# 지도를 통해서 인구수 시각화
# 지역마다 인구수가 어떻게 분포되어있는지 확인->지번을 통해서
# 

In [ ]:
# 지역마다 인구수가 어떻게 분포되어있는지 확인->지번을 통해서
# pip install folium
import folium

m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)
m